# Temperature采样的数学原理与可视化

## 1. 理论背景

在序列生成任务中，模型输出的是logits（未归一化的对数概率）。标准的softmax函数将这些logits转换为概率分布。

**标准Softmax函数**:
$$P(i) = \frac{\exp(z_i)}{\sum_j \exp(z_j)}$$

**Temperature Softmax函数**:
$$P_T(i) = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}$$

其中：
- $z_i$ 是第 $i$ 个类别的logit
- $T$ 是温度参数
- $P_T(i)$ 是经过温度缩放后的概率

## 2. 温度参数的影响

- **$T \to 0$**: 概率分布变得极度尖锐，接近one-hot分布（贪婪采样）
- **$T = 1$**: 保持模型原始输出的概率分布
- **$T > 1$**: 概率分布变得平滑，增加低概率项的采样机会
- **$T \to \infty$**: 概率分布趋近均匀分布（完全随机）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

## 7. 总结与实践建议

### 核心要点

1. **温度参数的本质**：调整概率分布的"锐度"，控制模型输出的确定性与多样性之间的权衡

2. **数学性质**：
   - 温度缩放是在logit空间进行的线性变换
   - 等价于在概率空间进行指数变换后重新归一化
   - 不改变概率的相对顺序，只改变其差距

3. **信息论视角**：
   - 温度控制输出分布的熵
   - 低温 = 低熵 = 低不确定性 = 高确定性
   - 高温 = 高熵 = 高不确定性 = 高随机性

### 实际应用指南

| 任务类型 | 推荐温度范围 | 说明 |
|---------|-------------|------|
| 机器翻译 | 0.3 - 0.7 | 需要高准确性，避免幻觉 |
| 代码生成 | 0.2 - 0.5 | 语法严格，错误代价高 |
| 摘要生成 | 0.5 - 0.8 | 平衡准确性和表达多样性 |
| 对话系统 | 0.7 - 1.0 | 需要自然和多样的回复 |
| 创意写作 | 0.8 - 1.5 | 鼓励创造性和新颖性 |
| 故事续写 | 1.0 - 2.0 | 追求意外和想象力 |

### 调优技巧

1. **起始建议**：从T=0.7开始，根据生成质量调整
2. **A/B测试**：对比不同温度下的生成样本，选择最佳值
3. **动态调整**：生成过程中可以动态改变温度（如开始低温，后期高温）
4. **与其他技术结合**：配合Top-k、Top-p（nucleus sampling）使用效果更佳

### 常见陷阱

- ❌ **过低温度** (T<0.1)：输出过于重复和单调
- ❌ **过高温度** (T>3.0)：输出不连贯，出现语法错误
- ❌ **忽视任务特性**：不同任务需要不同温度策略
- ✅ **实验验证**：始终通过实验验证温度参数的效果

In [ ]:
# 模拟一个简化的词汇表
vocabulary = ['the', 'a', 'is', 'are', 'cat', 'dog', 'beautiful', 'runs', 'quickly', 'slowly',
              'happy', 'sad', 'very', 'extremely', 'and', 'but', 'or', 'with', 'without', 'in']

# 模拟模型输出的logits（假设"the"和"a"概率较高）
simulated_logits = np.array([3.0, 2.5, 1.0, 0.8, 1.5, 1.2, 0.5, 0.6, 0.3, 0.2,
                              0.4, 0.1, 0.7, 0.0, 1.1, 0.9, 0.4, 0.8, 0.3, 0.6])

def sample_words(logits, vocab, temperature, num_samples=100):
    """
    从给定的logits分布中采样单词
    
    参数:
        logits: 模型输出的logits
        vocab: 词汇表
        temperature: 温度参数
        num_samples: 采样次数
    返回:
        采样结果的频率分布
    """
    probs = temperature_softmax(logits, temperature)
    
    # 进行多次采样
    samples = np.random.choice(len(vocab), size=num_samples, p=probs)
    
    # 统计频率
    unique, counts = np.unique(samples, return_counts=True)
    frequencies = np.zeros(len(vocab))
    frequencies[unique] = counts / num_samples
    
    return frequencies, probs


# 测试不同温度下的采样
test_temps = [0.2, 1.0, 2.0]
num_samples = 1000

fig, axes = plt.subplots(len(test_temps), 1, figsize=(14, 12))

for idx, temp in enumerate(test_temps):
    sample_freq, theory_probs = sample_words(simulated_logits, vocabulary, temp, num_samples)
    
    x = np.arange(len(vocabulary))
    width = 0.35
    
    # 理论概率与实际采样频率对比
    axes[idx].bar(x - width/2, theory_probs, width, label='理论概率', alpha=0.8, color='steelblue')
    axes[idx].bar(x + width/2, sample_freq, width, label='采样频率', alpha=0.8, color='coral')
    
    axes[idx].set_title(f'温度 T={temp} (采样{num_samples}次)', fontsize=13, fontweight='bold')
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(vocabulary, rotation=45, ha='right')
    axes[idx].set_ylabel('概率/频率')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)
    
    # 显示top-3采样词
    top3_indices = np.argsort(sample_freq)[-3:][::-1]
    top3_words = [vocabulary[i] for i in top3_indices]
    top3_freqs = [sample_freq[i] for i in top3_indices]
    
    info_text = f"Top-3采样词: {', '.join([f'{w}({f:.2%})' for w, f in zip(top3_words, top3_freqs)])}"
    axes[idx].text(0.02, 0.95, info_text, transform=axes[idx].transAxes,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.savefig('sampling_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\\n分析：")
print("- T=0.2: 采样高度集中在概率最高的词（'the', 'a'），生成文本单调")
print("- T=1.0: 采样分布与理论概率一致，平衡了确定性和多样性")
print("- T=2.0: 采样更加分散，低概率词也有机会被选中，生成文本更具创造性")

## 6. 实际应用：文本生成示例

演示在实际文本生成场景中，不同温度参数如何影响采样结果

In [ ]:
def compute_entropy(probs):
    """计算概率分布的香农熵"""
    return -np.sum(probs * np.log(probs + 1e-10))


# 测试温度与熵的关系
temperature_range = np.linspace(0.1, 5.0, 50)
entropies = []
max_probs = []

for temp in temperature_range:
    probs = temperature_softmax(logits, temp)
    entropies.append(compute_entropy(probs))
    max_probs.append(np.max(probs))

# 绘制关系图
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 熵随温度变化
ax1.plot(temperature_range, entropies, linewidth=2.5, color='darkred')
ax1.axvline(x=1.0, color='gray', linestyle='--', alpha=0.5, label='T=1 (标准)')
ax1.set_xlabel('温度参数 T', fontsize=12)
ax1.set_ylabel('熵 (nats)', fontsize=12)
ax1.set_title('熵与温度的关系', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)
ax1.legend()

# 最大概率随温度变化
ax2.plot(temperature_range, max_probs, linewidth=2.5, color='darkblue')
ax2.axvline(x=1.0, color='gray', linestyle='--', alpha=0.5, label='T=1 (标准)')
ax2.set_xlabel('温度参数 T', fontsize=12)
ax2.set_ylabel('最大概率', fontsize=12)
ax2.set_title('最大概率与温度的关系', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.savefig('entropy_temperature_relationship.png', dpi=150, bbox_inches='tight')
plt.show()

print("\\n关键发现：")
print(f"- 低温(T=0.1)时，熵={entropies[0]:.3f}，最大概率={max_probs[0]:.3f}")
print(f"- 标准(T=1.0)时，熵={entropies[np.argmin(np.abs(temperature_range-1.0))]:.3f}，"
      f"最大概率={max_probs[np.argmin(np.abs(temperature_range-1.0))]:.3f}")
print(f"- 高温(T=5.0)时，熵={entropies[-1]:.3f}，最大概率={max_probs[-1]:.3f}")
print(f"- 均匀分布的理论最大熵: {np.log(vocab_size):.3f}")

## 5. 熵与温度的关系

温度参数直接影响概率分布的熵。熵是衡量不确定性的指标，熵越高表示分布越均匀，不确定性越大。

**香农熵的定义**:
$$H(P) = -\sum_i P(i) \log P(i)$$

In [ ]:
# 创建一个模拟的logits分布
vocab_size = 20
logits = np.random.randn(vocab_size) * 2  # 模拟模型输出
logits = np.sort(logits)[::-1]  # 降序排列以便观察

# 测试不同的温度参数
temperatures = [0.1, 0.5, 1.0, 2.0, 5.0]

# 可视化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, temp in enumerate(temperatures):
    probs = temperature_softmax(logits, temp)
    
    axes[idx].bar(range(vocab_size), probs, alpha=0.7, color='steelblue')
    axes[idx].set_title(f'Temperature T = {temp}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('词汇索引 (按logit降序排列)')
    axes[idx].set_ylabel('概率')
    axes[idx].grid(axis='y', alpha=0.3)
    
    # 添加统计信息
    entropy = -np.sum(probs * np.log(probs + 1e-10))
    max_prob = np.max(probs)
    axes[idx].text(0.02, 0.95, f'熵: {entropy:.3f}\\n最大概率: {max_prob:.3f}',
                   transform=axes[idx].transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 隐藏多余的子图
axes[-1].axis('off')

plt.tight_layout()
plt.savefig('temperature_effect.png', dpi=150, bbox_inches='tight')
plt.show()

print("\\n观察：")
print("- T=0.1: 分布极度尖锐，几乎所有概率集中在最高logit项")
print("- T=0.5: 分布较尖锐，主要概率集中在前几项")
print("- T=1.0: 标准分布，保持模型原始输出")
print("- T=2.0: 分布变平滑，低概率项机会增加")
print("- T=5.0: 分布接近均匀，几乎是随机采样")

## 4. 可视化：温度参数的影响

通过可视化展示不同温度参数如何改变概率分布的形状

In [ ]:
def softmax(logits):
    """
    标准softmax函数
    
    参数:
        logits: 未归一化的对数概率
    返回:
        概率分布
    """
    exp_logits = np.exp(logits - np.max(logits))  # 减去最大值以数值稳定
    return exp_logits / np.sum(exp_logits)


def temperature_softmax(logits, temperature=1.0):
    """
    带温度参数的softmax函数
    
    参数:
        logits: 未归一化的对数概率
        temperature: 温度参数
            - T < 1: 使分布更尖锐
            - T = 1: 标准softmax
            - T > 1: 使分布更平滑
    返回:
        经过温度缩放的概率分布
    """
    scaled_logits = logits / temperature
    return softmax(scaled_logits)


def reweight_distribution(original_probs, temperature=1.0):
    """
    对已有的概率分布进行温度重加权（另一种实现方式）
    
    该方法通过对概率取对数转回logit空间，应用温度缩放后再转回概率空间
    
    参数:
        original_probs: 原始概率分布（和为1）
        temperature: 温度参数
    返回:
        重加权后的概率分布
    """
    # 转回logit空间
    logits = np.log(original_probs + 1e-10)  # 加小常数避免log(0)
    
    # 温度缩放
    scaled_logits = logits / temperature
    
    # 转回概率空间
    exp_logits = np.exp(scaled_logits)
    return exp_logits / np.sum(exp_logits)


# 测试两种实现的等价性
test_logits = np.array([2.0, 1.0, 0.1])
test_probs = softmax(test_logits)
test_temp = 0.5

method1 = temperature_softmax(test_logits, test_temp)
method2 = reweight_distribution(test_probs, test_temp)

print("原始logits:", test_logits)
print("原始概率分布:", test_probs)
print(f"\n温度T={test_temp}时:")
print("方法1 (temperature_softmax):", method1)
print("方法2 (reweight_distribution):", method2)
print("两种方法的最大差异:", np.max(np.abs(method1 - method2)))

## 3. 核心函数实现

实现两种版本的temperature采样函数：
1. 基于概率分布的重加权
2. 基于logits的直接缩放